In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import timedelta
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

from torch.utils.data import DataLoader, Dataset
from transformers import LongformerModel, LongformerTokenizer

c:\Anaconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Anaconda3\envs\myenv\Lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Anaconda3\envs\myenv\Lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
# 데이터 로드
stock_df = pd.read_excel('../../../data/tb_stock.xlsx')
main_economic_df = pd.read_excel('../../../data/tb_main_economic_index.xlsx')
korea_economic_df = pd.read_excel('../../../data/tb_korea_economic_indicator.xlsx')

#stock_df  = stock_df[-365:]
#main_economic_df  = main_economic_df[-365:]
#korea_economic_df  = korea_economic_df[-365:]

# 필요한 열만 선택
stock_df = stock_df[['sc_date', 'sc_ss_stock']]
main_economic_df = main_economic_df[['mei_date', 'mei_gold', 'mei_sp500', 'mei_kospi']]
korea_economic_df = korea_economic_df[['kei_date', 'kei_m2_avg', 'kei_fr']]

# 열 이름 변경
stock_df.rename(columns={'sc_date': 'date'}, inplace=True)
main_economic_df.rename(columns={'mei_date': 'date'}, inplace=True)
korea_economic_df.rename(columns={'kei_date': 'date'}, inplace=True)

# 데이터프레임 병합
merged_df = pd.merge(stock_df, main_economic_df, on='date', how='inner')
merged_df = pd.merge(merged_df, korea_economic_df, on='date', how='inner')

In [3]:
# 텍스트 데이터 생성
merged_df['text'] = merged_df.apply(lambda row: f"On {row['date']}, gold price was {row['mei_gold']}, S&P 500 index was {row['mei_sp500']}, KOSPI index was {row['mei_kospi']}, M2 average was {row['kei_m2_avg']}, and FR was {row['kei_fr']}.", axis=1)

# 모델의 타겟 설정
merged_df['target'] = merged_df['sc_ss_stock']

# 날짜 형식 확인 및 변환
if not pd.api.types.is_datetime64_any_dtype(merged_df['date']):
         merged_df['date'] = pd.to_datetime(merged_df['date'])

print(merged_df[['date', 'text', 'target']].head())

        date                                               text  target
0 2014-09-17  On 2014-09-17, gold price was 1234.40002441406...   24520
1 2014-09-18  On 2014-09-18, gold price was 1225.69995117187...   24200
2 2014-09-19  On 2014-09-19, gold price was 1215.30004882812...   24200
3 2014-09-20  On 2014-09-20, gold price was 1215.30004882812...   24200
4 2014-09-21  On 2014-09-21, gold price was 1215.30004882812...   24200


In [4]:
# 1. 데이터셋 정의
class StockDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length                              # 토큰화 사용시 최대 시퀀스 길이를 저장

    def __len__(self):                                            # 데이터의 개수를 반환
        return len(self.data)

    def __getitem__(self, idx):                                   # text, target 반환
        text   = self.data.iloc[idx]['text']
        target = self.data.iloc[idx]['target']
        
        inputs = self.tokenizer(text, 
                                truncation=True,                  # 시퀀스가 max_length를 초과할 경우 자릅니다.
                                padding   ='max_length',          # 시퀀스가 max_length에 맞게 패딩
                                max_length=self.max_length,       # 시퀀스 최대길이 설정
                                return_tensors="pt")              # 결과를 pytorch 텐서 형식으로 반환
        
        return {
            'input_ids': inputs['input_ids'].flatten(),           # 토큰 ID 배열을 반환
            'attention_mask': inputs['attention_mask'].flatten(), # 패딩된 토큰을 무시하기 위한 마스크 배열 반환
            'target': torch.tensor(target, dtype=torch.float)     # 타겟ㅇ르 float 타입의 텐서로 반환
        }

In [5]:
# 2. Pruning을 적용하는 모델 정의
class PrunedStockPricePredictor(nn.Module):
    # 클래스 초기화 메서드
    def __init__(self, longformer_model_name):
        # super은 부모 혹은 상위 클래스를 호출하여 기능을 사용하기 위해 쓰일 수 있다.
        super(PrunedStockPricePredictor, self).__init__()
        # 사전 학습된 longformermodel을 불러올 수 있다.
        self.longformer = LongformerModel.from_pretrained(longformer_model_name)
        # Fully Connected Layer으로 벡터화, 특징 추출의 마지막 단계이다.
        self.fc         = nn.Linear(self.longformer.config.hidden_size, 1)

    # 모델의 순전파 과정을 정의
    def forward(self, input_ids, attention_mask):
        outputs = self.longformer(input_ids=input_ids, attention_mask=attention_mask)
        # 모델의 출력에서 첫 번째 토큰([CLS] 토큰)의 벡터를 가져온다.
        # outputs[0]은 마지막 레이어의 은닉 상태 텐서를 의미하며, 모든 토큰에 대한 정보를 포함
        # 크키 : [batch_size, sequence_length, hidden_size]
        cls_output = outputs[0][:, 0, :]
        return self.fc(cls_output)
    
    def apply_pruning(self, pruning_amount=0.4):
        # Fully connected layer에 L1 가지치기 적용
        # 가지치기를 적용할 대상이 가중치(weight)
        prune.l1_unstructured(self.fc, name="weight", amount=pruning_amount)
        # 가지치기 적용 후 pruned 상태에서 재학습을 위해 제거
        prune.remove(self.fc, 'weight')

In [6]:
# 3. 데이터 준비
tokenizer         = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
max_length        = 512

train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)
train_dataset     = StockDataset(train_df, tokenizer, max_length)
test_dataset      = StockDataset(test_df, tokenizer, max_length)

# 데이터셋을 배치 단위로 로드
# 학습 시에는 무작위로 일반화 능력을 높이고,
# 테스트 시에는 섞지 않아 일관된 평가를 보장
train_loader      = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader       = DataLoader(test_dataset, batch_size=4, shuffle=False)

c:\Anaconda3\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
# 4. 데이터 분할 및 변수 초기화

X_train = np.array([
    tokenizer(text, truncation=True, padding='max_length', max_length=max_length, 
              return_tensors='pt')
              ['input_ids'].        # 'input_ids'로 반환(토큰의 ID)
              flatten().            # 2차원 텐서로 변환 [batch_size, sequence_length]의 형태
              numpy()               # PyTorch 텐서를 NumPy 배열로 변환
    for text in train_df['text']
])

y_train = train_df['target'].values

# 훈련 데이터와 동일한 방식으로 토큰화 및 패딩을 수행
X_val = np.array([
    tokenizer(text, truncation=True, padding='max_length', max_length=max_length, 
              return_tensors='pt')
              ['input_ids'].        # 'input_ids'로 반환(토큰의 ID)
              flatten().            # 2차원 텐서로 변환 [batch_size, sequence_length]의 형태
              numpy()               # PyTorch 텐서를 NumPy 배열로 변환
    for text in train_df['text']
])

y_val = train_df['target'].values

# 모델 저장
best_model_path = '../saved_models/EconomicPredict_Smsung_Longformer_best.pt'

# 모델 저장 경로의 디렉토리를 생성합니다.
# os.makedirs는 디렉토리가 없을 경우 생성하며, exist_ok=True로 이미 존재하는 경우 에러를 방지합니다.
os.makedirs(os.path.dirname(best_model_path), exist_ok=True)

# 파라미터 리스트
learning_rates = [1e-5, 3e-5, 5e-5]
model_names = ['allenai/longformer-base-4096', 'allenai/longformer-large-4096']

# 최적의 모델을 찾기 위해 초기 최적 점수를 무한대로 설정
# 이 변수는 나중에 검증 손실이 현재 최적 점수보다 낮은 경우 업데이트
best_score = float('inf')

# 학습과 정에서 발생하는 훈련 손실 값을 저장하는 리스트
train_losses = []

# 학습 과정에서 발생하는 검증 손실 값을 저장할 리스트
val_losses   = []

In [8]:
# 5. 최적의 하이퍼 파라미터 찾기
for lr in tqdm(learning_rates, desc='최고의 학습률'):
    for model_name in tqdm(model_names, desc='최고의 모델'):
        print(f"Training with lr={lr}, model_name={model_name}")

        # 모델 초기화
        model = PrunedStockPricePredictor(model_name)
        model.apply_pruning(pruning_amount=0.4)
        model.train()

        # Adam은 각 파라미터에 대해 개별 학습률을 유지하고 업데이트하며, 학습 중에 학습률을 조정
        # 기울기 값을 고려하면서 파라미터를 업데이트
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # 손실 함수로 MSE를 사용 (평균 제곱 오차)
        criterion = nn.MSELoss()

        train_loss_per_epoch = 0
        val_loss_per_epoch = 0

        for epoch in tqdm(range(3)):
            model.train()

            # 현재 에포크 동안의 누적 훈련 손실을 저장할 변수를 초기화
            running_loss = 0.0

            # 훈련 데이터셋을 배치 단위로 나누어 학습
            for i in range(0, len(X_train), 4):
                # 입력 데이터와 타깃 데이터를 텐서로 변환, 모델이 사용 중인 디바이스로 전환
                input_ids = torch.tensor(X_train[i:i+4]).to(model.longformer.device)
                attention_mask = (input_ids != 0).long().to(model.longformer.device)
                targets = torch.tensor(y_train[i:i+4], dtype=torch.float).to(model.longformer.device)
                
                # 옵티마이저 기울기를 초기화
                optimizer.zero_grad()

                # 모델을 통해 예측값을 계산
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)

                # 에측값과 실제값 사이의 손실(loss)을 계산합니다.
                loss = criterion(outputs.squeeze(), targets)

                # 역전파를 통해 기울기를 계산하고, 옵티마이저가 이를 기반으로 파라미터를 업데이트
                loss.backward()
                optimizer.step()

                # 현재 배치의 손실을 누적
                running_loss += loss.item()

            # 에포크가 끝날 때마다 평균 훈련 손실을 계산하고, 리스트에 추가
            train_loss_per_epoch = running_loss / len(X_train) * 4
            train_losses.append(train_loss_per_epoch)

            # 검증 단계
            model.eval()
            val_loss = 0.0        # 현재 에포크 동안의 누적 검증 손실을 저장할 변수
            with torch.no_grad(): # 평가단계에서는 기울기를 계산하지 않는다.
                for i in range(0, len(X_val), 4):
                    # 입력, 타겟 데이터를 텐서로 변환하고,  모델이 사용 중인 디바이스로 전송
                    input_ids = torch.tensor(X_val[i:i+4]).to(model.longformer.device)
                    attention_mask = (input_ids != 0).long().to(model.longformer.device)
                    targets = torch.tensor(y_val[i:i+4], dtype=torch.float).to(model.longformer.device)
                    
                    # 검증 데이터에 대한 예측값을 계산합니다.
                    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

                    # 예측값과 실제값 사이의 손실을 계산
                    loss = criterion(outputs.squeeze(), targets)
                    val_loss += loss.item()
            
            val_loss_per_epoch = val_loss / len(X_val) * 4
            val_losses.append(val_loss_per_epoch)

            print(f'Epoch {epoch+1}: Training Loss: {train_loss_per_epoch}, Validation Loss: {val_loss_per_epoch}')

        # 최적의 모델 저장
        if val_loss_per_epoch < best_score:
            best_score = val_loss_per_epoch
            torch.save(model.state_dict(), best_model_path)

print(f"Best Validation Loss: {best_score}")
print(f"Best model saved to {best_model_path}")

최고의 학습률:   0%|          | 0/3 [00:00<?, ?it/s]

Training with lr=1e-05, model_name=allenai/longformer-base-4096


c:\Anaconda3\envs\myenv\Lib\site-packages\transformers\modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=m

Epoch 1: Training Loss: 3095903484.1104975, Validation Loss: 3095573606.8066297


In [ ]:
# 모델을 평가 모드로 설정
# 드룹아웃, 배치 정규화 등 레이어들을 비활성화
model.eval()
y_train_pred = []
y_val_pred = []

with torch.no_grad():
    for i in range(0, len(X_train), 4):
        input_ids = torch.tensor(X_train[i:i+4]).to(model.longformer.device)
        # attention_mask는 입력 데이터에서 패딩 토큰(0)을 무시하는 역할
        attention_mask = (input_ids != 0).long().to(model.longformer.device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        y_train_pred.extend(outputs.cpu().numpy())

    for i in range(0, len(X_val), 4):
        input_ids = torch.tensor(X_val[i:i+4]).to(model.longformer.device)
        attention_mask = (input_ids != 0).long().to(model.longformer.device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        y_val_pred.extend(outputs.cpu().numpy())

In [ ]:
# 평가 지표 계산
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)

val_mse = mean_squared_error(y_val, y_val_pred)
val_rmse = np.sqrt(val_mse)
val_mae = mean_absolute_error(y_val, y_val_pred)

print(f"Training MSE: {train_mse}, RMSE: {train_rmse}, MAE: {train_mae}")
print(f"Validation MSE: {val_mse}, RMSE: {val_rmse}, MAE: {val_mae}")

In [ ]:
# 7. 과적합 여부 확인
if min(val_mse, train_mse) < min(train_mse, val_mse):
    print("경고: 과적합이 발생할 가능성이 있습니다. 검증 손실이 훈련 손실보다 낮습니다.")
elif val_rmse > train_rmse:
    print("경고: 과적합 가능성이 있습니다. 검증 손실이 훈련 손실보다 높습니다.")
else:
    print("모델이 적절하게 학습되었습니다. 과적합이 크게 발생하지 않았습니다.")

In [ ]:
# 훈련 및 검증 손실의 변화 추이 그래프
plt.figure(figsize=(14, 7))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
plt.title('Training and Validation Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# 잔차 계산
train_residuals = y_train - np.array(y_train_pred).flatten()
val_residuals = y_val - np.array(y_val_pred).flatten()

# 잔차 플롯 (훈련 데이터)
plt.figure(figsize=(10, 6))
plt.scatter(y_train_pred, train_residuals, alpha=0.5, label='Train Residuals')
plt.hlines(0, min(y_train_pred), max(y_train_pred), color='black', linestyle='--')
plt.title('Residuals vs Predicted (Train Data)')
plt.xlabel('Predicted Prices')
plt.ylabel('Residuals')
plt.legend()
plt.show()

# 잔차 플롯 (검증 데이터)
plt.figure(figsize=(10, 6))
plt.scatter(y_val_pred, val_residuals, alpha=0.5, label='Validation Residuals', color='red')
plt.hlines(0, min(y_val_pred), max(y_val_pred), color='black', linestyle='--')
plt.title('Residuals vs Predicted (Validation Data)')
plt.xlabel('Predicted Prices')
plt.ylabel('Residuals')
plt.legend()
plt.show()

In [ ]:
# 예측 오차 히스토그램 (훈련 데이터)
plt.figure(figsize=(10, 6))
plt.hist(train_residuals, bins=50, alpha=0.5, label='Train Residuals', color='blue')
plt.title('Distribution of Prediction Errors (Train Data)')
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# 예측 오차 히스토그램 (검증 데이터)
plt.figure(figsize=(10, 6))
plt.hist(val_residuals, bins=50, alpha=0.5, label='Validation Residuals', color='red')
plt.title('Distribution of Prediction Errors (Validation Data)')
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
# 시계열 그래프 (훈련 데이터)
plt.figure(figsize=(14, 7))
plt.plot(range(len(y_train)), y_train, label='Actual Train Prices', color='blue')
plt.plot(range(len(y_train_pred)), y_train_pred, label='Predicted Train Prices', color='red', linestyle='--')
plt.title('Actual vs Predicted Prices on Training Data')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

# 시계열 그래프 (검증 데이터)
plt.figure(figsize=(14, 7))
plt.plot(range(len(y_val)), y_val, label='Actual Validation Prices', color='blue')
plt.plot(range(len(y_val_pred)), y_val_pred, label='Predicted Validation Prices', color='red', linestyle='--')
plt.title('Actual vs Predicted Prices on Validation Data')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()